Configurations
- dataset: vaso_eps_0_1
- annotation function: annotOpt

In [2]:
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['FreeSans']
%config InlineBackend.figure_formats = ['svg']

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pickle
import itertools
import copy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn import metrics
import itertools

import joblib
from joblib import Parallel, delayed

In [4]:
from OPE_utils_new import (
    format_data_tensor,
    policy_eval_analytic_finite,
    OPE_IS_h,
    compute_behavior_policy_h,
)

In [5]:
NSTEPS = H = 20   # max episode length in historical data # Horizon of the MDP
G_min = -1        # the minimum possible return
G_max =  1        # the maximum possible return
nS, nA = 1442, 8

PROB_DIAB = 0.2

In [6]:
# Ground truth MDP model
MDP_parameters = joblib.load('../data/MDP_parameters.joblib')
P = MDP_parameters['transition_matrix_absorbing'] # (A, S, S_next)
R = MDP_parameters['reward_matrix_absorbing_SA'] # (S, A)
nS, nA = R.shape
gamma = 0.99

# unif rand isd, mixture of diabetic state
isd = joblib.load('../data/modified_prior_initial_state_absorbing.joblib')
isd = (isd > 0).astype(float)
isd[:720] = isd[:720] / isd[:720].sum() * (1-PROB_DIAB)
isd[720:] = isd[720:] / isd[720:].sum() * (PROB_DIAB)

In [7]:
# Precomputed optimal policy
π_star = joblib.load('../data/π_star.joblib')

## Load data

In [8]:
input_dir = '../datagen/vaso_eps_0_1-100k/'

In [9]:
def load_data(fname):
    print('Loading data', fname, '...', end='')
    df_data = pd.read_csv('{}/{}'.format(input_dir, fname)).rename(columns={'State_idx': 'State'})#[['pt_id', 'Time', 'State', 'Action', 'Reward']]

    # Assign next state
    df_data['NextState'] = [*df_data['State'].iloc[1:].values, -1]
    df_data.loc[df_data.groupby('pt_id')['Time'].idxmax(), 'NextState'] = -1
    df_data.loc[(df_data['Reward'] == -1), 'NextState'] = 1440
    df_data.loc[(df_data['Reward'] == 1), 'NextState'] = 1441

    assert ((df_data['Reward'] != 0) == (df_data['Action'] == -1)).all()

    print('DONE')
    return df_data

In [10]:
df_seed1 = load_data('1-features.csv') # tr
df_seed2 = load_data('2-features.csv') # va

Loading data 1-features.csv ...DONE
Loading data 2-features.csv ...DONE


## Policies

### Behavior policy

In [11]:
# vaso eps=0.5, mv abx optimal
π_beh = (np.tile(π_star.reshape((-1,2,2,2)).sum(axis=3, keepdims=True), (1,1,1,2)).reshape((-1, 8)) / 2)
π_beh[π_star == 1] = 0.9
π_beh[π_beh == 0.5] = 0.1

In [12]:
V_H_beh = policy_eval_analytic_finite(P.transpose((1,0,2)), R, π_beh, gamma, H)
Q_H_beh = [(R + gamma * P.transpose((1,0,2)) @ V_H_beh[h]) for h in range(1,H)] + [R]

In [13]:
J_beh = isd @ V_H_beh[0]
J_beh

0.2503835479385116

In [14]:
# Check recursive relationships
assert len(Q_H_beh) == H
assert len(V_H_beh) == H
assert np.all(Q_H_beh[-1] == R)
assert np.all(np.sum(π_beh * Q_H_beh[-1], axis=1) == V_H_beh[-1])
assert np.all(R + gamma * P.transpose((1,0,2)) @ V_H_beh[-1] == Q_H_beh[-2])

### Evaluation policy

In [15]:
π_eval = π_star

In [16]:
V_H_eval = policy_eval_analytic_finite(P.transpose((1,0,2)), R, π_eval, gamma, H)
Q_H_eval = [(R + gamma * P.transpose((1,0,2)) @ V_H_eval[h]) for h in range(1,H)] + [R]

In [17]:
J_eval = isd @ V_H_eval[0]
J_eval

0.40877179296760235

In [18]:
# Check recursive relationships
assert len(Q_H_eval) == H
assert len(V_H_eval) == H
assert np.all(Q_H_eval[-1] == R)
assert np.all(np.sum(π_eval * Q_H_eval[-1], axis=1) == V_H_eval[-1])
assert np.all(R + gamma * P.transpose((1,0,2)) @ V_H_eval[-1] == Q_H_eval[-2])

## Pre-populate counterfactual annotations for offline dataset

### version 1: annotate counterfactuals only for initial states

In [19]:
df_va = df_seed2[['pt_id', 'Time', 'State', 'Action', 'Reward', 'NextState']].copy()

# assign alternative action for vaso
df_va['Action:Vaso'] = df_va['Action'] % 2
df_va.loc[df_va['Action'] == -1, 'Action:Vaso'] = -1
df_va['Action_Alt'] = df_va['Action'] + 1 - 2*df_va['Action:Vaso']
df_va.loc[df_va['Action'] == -1, 'Action_Alt'] = -1

In [20]:
# for each original traj, create a length 1 pseudo-traj by flipping the action from starting state
def _func_v1(df_i):
    df_i_new = []
    for t in range(len(df_i) - 1):
        if t > 0: break
        step, S, A_alt = df_i.iloc[t]['Time'], df_i.iloc[t]['State'], df_i.iloc[t]['Action_Alt']
        df_i_t = df_i.iloc[:t].loc[:, ['Time', 'State', 'Action', 'Reward', 'NextState']].append(
            pd.Series({
                'Time': step,
                'State': S,
                'Action': A_alt,
                'Reward': Q_H_eval[t][S, A_alt],
                'NextState': 1442, # truncation indicator
            }), ignore_index=True,
        )
        df_i_t['pt_id'] = df_i['pt_id'].iloc[0] + (t+1)*0.01
        df_i_t = df_i_t[['pt_id', 'Time', 'State', 'Action', 'Reward', 'NextState']]
        df_i_new.append(df_i_t)
    return df_i_new

In [21]:
df_va_new1 = Parallel(n_jobs=50)(delayed(_func_v1)(df_i) for i, df_i in tqdm(df_va.groupby('pt_id')))
df_va_new1 = pd.concat(itertools.chain.from_iterable(df_va_new1)).reset_index(drop=True)
df_va_new1[['Time', 'State', 'Action', 'NextState']] = df_va_new1[['Time', 'State', 'Action', 'NextState']].astype(int)

100%|██████████| 100000/100000 [13:01<00:00, 128.00it/s]


In [22]:
df_va_all1 = pd.concat([
    df_va[['pt_id', 'Time', 'State', 'Action', 'Reward', 'NextState']], 
    df_va_new1]).sort_values(by=['pt_id', 'Time']).reset_index(drop=True)

In [23]:
df_va_all1

,pt_id,Time,State,Action,Reward,NextState
0,200000.00,0,339,7,0.000000,463
1,200000.00,1,463,6,0.000000,381
2,200000.00,2,381,0,0.000000,376
3,200000.00,3,376,-1,1.000000,1441
4,200000.01,0,339,6,0.678555,1442
...,...,...,...,...,...,...
1475561,299999.00,16,365,6,0.000000,365
1475562,299999.00,17,365,6,0.000000,365
1475563,299999.00,18,365,6,0.000000,365
1475564,299999.00,19,365,6,0.000000,-1


In [24]:
df_va_all1.to_pickle('results/vaso_eps_0_1-evalOpt_df_seed2_aug_init.pkl')

### version 2: annotate counterfactuals for all time steps

In [25]:
df_va = df_seed2[['pt_id', 'Time', 'State', 'Action', 'Reward', 'NextState']].copy()

# assign alternative action for vaso
df_va['Action:Vaso'] = df_va['Action'] % 2
df_va.loc[df_va['Action'] == -1, 'Action:Vaso'] = -1
df_va['Action_Alt'] = df_va['Action'] + 1 - 2*df_va['Action:Vaso']
df_va.loc[df_va['Action'] == -1, 'Action_Alt'] = -1

In [26]:
def _func_v2(df_i):
    df_i_new = []
    for t in range(len(df_i) - 1):
        step, S, A_alt = df_i.iloc[t]['Time'], df_i.iloc[t]['State'], df_i.iloc[t]['Action_Alt']
        df_i_t = df_i.iloc[:t].loc[:, ['Time', 'State', 'Action', 'Reward', 'NextState']].append(
            pd.Series({
                'Time': step,
                'State': S,
                'Action': A_alt,
                'Reward': Q_H_eval[t][S, A_alt],
                'NextState': 1442, # truncation indicator
            }), ignore_index=True,
        )
        df_i_t['pt_id'] = df_i['pt_id'].iloc[0] + (t+1)*0.01
        df_i_t = df_i_t[['pt_id', 'Time', 'State', 'Action', 'Reward', 'NextState']]
        df_i_new.append(df_i_t)
    return df_i_new

In [27]:
df_va_new2 = Parallel(n_jobs=100)(delayed(_func_v2)(df_i) for i, df_i in tqdm(df_va.groupby('pt_id')))
df_va_new2 = pd.concat(itertools.chain.from_iterable(df_va_new2)).reset_index(drop=True)
df_va_new2[['Time', 'State', 'Action', 'NextState']] = df_va_new2[['Time', 'State', 'Action', 'NextState']].astype(int)

100%|██████████| 100000/100000 [29:24<00:00, 56.69it/s]


In [28]:
df_va_all2 = pd.concat([
    df_va[['pt_id', 'Time', 'State', 'Action', 'Reward', 'NextState']], 
    df_va_new2]).sort_values(by=['pt_id', 'Time']).reset_index(drop=True)

In [ ]:
df_va_all2

,pt_id,Time,State,Action,Reward,NextState
0,200000.00,0,339,7,0.000000,463
1,200000.00,1,463,6,0.000000,381
2,200000.00,2,381,0,0.000000,376
3,200000.00,3,376,-1,1.000000,1441
4,200000.01,0,339,6,0.678555,1442
...,...,...,...,...,...,...
12463512,299999.19,14,365,6,0.000000,365
12463513,299999.19,15,365,6,0.000000,365
12463514,299999.19,16,365,6,0.000000,365
12463515,299999.19,17,365,6,0.000000,365


In [29]:
df_va_all2.to_pickle('results/vaso_eps_0_1-evalOpt_df_seed2_aug_step.pkl')